In [2]:
import sqlite3
import pandas as pd

from helpers import tap_request, clean_data

## Get planetary system data from archive

In [ ]:
service_url = "https://exoplanetarchive.ipac.caltech.edu/TAP"
ps_query = "SELECT pl_name, hostname, sy_snum, sy_pnum, cb_flag, disc_pubdate FROM ps"

ps_df = tap_request(service_url=service_url, query=ps_query, sync_type="async")
ps_df.to_csv('planetary_systems.csv', index=False)

In [8]:
sort_column = 'pl_name'

ps_df = pd.read_csv("planetary_systems.csv", index_col=False)
ps_df = clean_data(ps_df, sort_column)


#### see data frame

In [ ]:
ps_df

### basic append of new data if needed for testing

In [ ]:
# ps_df.to_sql('planetary_systems', conn, if_exists='append', index=False)

## Test dataframes

In [5]:
df_1 = pd.read_csv("tests\planetary_systems_1.csv", index_col=False)
df_2 = pd.read_csv("tests\planetary_systems_2.csv", index_col=False)

In [ ]:
df_1

In [ ]:
df_2

## connect

In [15]:
conn = sqlite3.connect("../database.db")
# conn = sqlite3.connect("test_database.db")
cursor = conn.cursor()

## Delete

In [ ]:
# conn.execute("""DROP TABLE IF EXISTS planetary_systems""")

In [ ]:
# conn.execute("""DROP TABLE IF EXISTS stellar_hosts""")

## Create

In [ ]:
conn.execute("""
CREATE TABLE planetary_systems (
    pl_name TEXT PRIMARY KEY NOT NULL,
    hostname TEXT,
    sy_snum INTEGER,
    sy_pnum INTEGER,
    cb_flag INTEGER,
    disc_pubdate INTEGER,
    last_updated DATETIME DEFAULT current_timestamp
);
""")

In [ ]:
conn.execute("""
CREATE TABLE stellar_hosts (
    sy_name TEXT PRIMARY KEY NOT NULL,
    sy_snum INTEGER,
    sy_pnum INTEGER,
    last_updated DATETIME DEFAULT current_timestamp
);
""")

## UPSERT into data base (WIP)

### Planetary Systems

In [35]:
# data_to_insert = []
# # for _, row in df_1.iterrows():
# # for _, row in df_2.iterrows():
# for _, row in ps_df.iterrows():
#     data_to_insert.append((
#         row['pl_name'],
#         row['hostname'],
#         row['sy_snum'],
#         row['sy_pnum'],
#         row['cb_flag'],
#         row['disc_pubdate']
#     ))

# conn.executemany(f"""
#     INSERT INTO planetary_systems (pl_name, hostname, sy_snum, sy_pnum, cb_flag, disc_pubdate, last_updated)
#     VALUES (?, ?, ?, ?, ?, ?, current_timestamp)
#     ON CONFLICT(pl_name)
#     DO UPDATE SET
#         hostname = CASE WHEN excluded.hostname != planetary_systems.hostname THEN excluded.hostname ELSE planetary_systems.hostname END,
#         sy_snum = CASE WHEN excluded.sy_snum != planetary_systems.sy_snum THEN excluded.sy_snum ELSE planetary_systems.sy_snum END,
#         sy_pnum = CASE WHEN excluded.sy_pnum != planetary_systems.sy_pnum THEN excluded.sy_pnum ELSE planetary_systems.sy_pnum END,
#         cb_flag = CASE WHEN excluded.cb_flag != planetary_systems.cb_flag THEN excluded.cb_flag ELSE planetary_systems.cb_flag END,
#         last_updated = current_timestamp
#     WHERE
#         planetary_systems.sy_snum != excluded.sy_snum OR
#         planetary_systems.sy_pnum != excluded.sy_pnum OR
#         planetary_systems.cb_flag != excluded.cb_flag OR
#         planetary_systems.hostname != excluded.hostname;
# """, data_to_insert)

### Stellar Hosts

In [11]:
# old_hosts = pd.read_csv("../static/20241206/stellar_hosts20241206.csv")

In [ ]:
# old_hosts

In [ ]:
# data_to_insert = []
# for _, row in old_hosts.iterrows():
#     data_to_insert.append((
#         row['sy_name'],
#         row['sy_snum'],
#         row['sy_pnum'],
#     ))

# conn.executemany(f"""
#     INSERT INTO stellar_hosts (sy_name, sy_snum, sy_pnum, last_updated)
#     VALUES (?, ?, ?, current_timestamp)
#     ON CONFLICT(sy_name)
#     DO UPDATE SET
#         sy_name = CASE WHEN excluded.sy_name != stellar_hosts.sy_name THEN excluded.sy_name ELSE stellar_hosts.sy_name END,
#         sy_snum = CASE WHEN excluded.sy_snum != stellar_hosts.sy_snum THEN excluded.sy_snum ELSE stellar_hosts.sy_snum END,
#         sy_pnum = CASE WHEN excluded.sy_pnum != stellar_hosts.sy_pnum THEN excluded.sy_pnum ELSE stellar_hosts.sy_pnum END,
#         last_updated = current_timestamp
#     WHERE
#         stellar_hosts.sy_snum != excluded.sy_snum OR
#         stellar_hosts.sy_pnum != excluded.sy_pnum;
# """, data_to_insert)

## check for tables

In [ ]:
pd.read_sql_query("""SELECT name FROM sqlite_master WHERE type = 'table'""", conn)

## check table metadata

In [ ]:
pd.read_sql_query("""PRAGMA table_info('planetary_systems');""", conn)

In [ ]:
pd.read_sql_query("""PRAGMA table_info('stellar_hosts');""", conn)

## Read planetary_systems

##### Get planet names from planetary_systems

In [ ]:
pd.read_sql_query("""SELECT * FROM planetary_systems""", conn)

##### Get count from planetary_systems

In [ ]:
pd.read_sql_query("SELECT COUNT(pl_name) FROM planetary_systems", conn)

##### Get all from stellar_hosts

In [ ]:
pd.read_sql_query("""SELECT * FROM stellar_hosts""", conn)

##### Get count from stellar_hosts

In [ ]:
pd.read_sql_query("SELECT COUNT(sy_name) FROM stellar_hosts", conn)

##### Get most recent updated stellar_hosts

In [ ]:
pd.read_sql_query("""
SELECT * FROM stellar_hosts
 WHERE last_updated = (
  SELECT MAX(last_updated)
    FROM stellar_hosts
);""", conn)

##### Get all from planetary_systems by stellar_hosts specific system name

In [ ]:
pd.read_sql_query("""
SELECT *
FROM   planetary_systems
WHERE  hostname=(
  SELECT sy_name 
  FROM   stellar_hosts
  WHERE  sy_name='14 Her'
  )
""", conn)

## Commit and Close connection

In [25]:
conn.commit()
conn.close()